# Perplexity playbook

<p>
Mal Minhas, v0.4<br>
02.06.24
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.3</b>: 15.01.24. First version</li>
<li><b>v0.4</b>: 02.06.24. Updated for latest Perplexity API</li>
</ul>
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Perplexity Models
Model	Parameter Count	Context Length	Model Type
llama-3-sonar-small-32k-chat	8B	32768	Chat Completion
llama-3-sonar-small-32k-online	8B	28000	Chat Completion
llama-3-sonar-large-32k-chat	70B	32768	Chat Completion
llama-3-sonar-large-32k-online	70B	28000	Chat Completion

Open-Source Models
Where possible, we try to match the Hugging Face implementation.
Model	Parameter Count	Context Length	Model Type
llama-3-8b-instruct	8B	8192	Chat Completion
llama-3-70b-instruct	70B	8192	Chat Completion
mixtral-8x7b-instruct	8x7B	16384	Chat Completion
```

Of these models, the response [here](https://docs.perplexity.ai/discuss/65b27c5fc46f38007527c07f) suggests you should use `pplx-7b-online` or `pplx-70b-online` for real-time data.

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import Markdown, display

def renderCompletion(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get("PERPLEXITY_API_KEY")
DEFAULT_MODEL = "mistral-7b-instruct"
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = "Be precise and concise."
MID_LEVEL_DETAIL = "Explain in a few sentences."
GREAT_DETAIL = "Explain in a lot of detail."
POLITE_DETAIL = "You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user."
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return renderCompletion(r.json().get("choices")[0].get("message").get("content"))

Let's test the function on an example query using the default Mistral 7b model and then Llama2.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
getPerplexityResponse(prompt)

In Southampton, UK, if you have a broken washing machine, you should call a professional appliance repair technician. They have the necessary skills and expertise to diagnose and fix common washing machine issues. You can look up local appliance repair services online or check directories like Yell or Thomson Local. Alternatively, you could contact a local appliance repair company, such as Southampton Appliance Repairs or A1 Appliance Repairs, for assistance. Make sure to describe the problem accurately when you call to help them prepare for the repair visit.

CPU times: user 38.7 ms, sys: 10.5 ms, total: 49.2 ms
Wall time: 1.38 s


In [4]:
%%time

getPerplexityResponse(prompt, model='llama-3-8b-instruct')

Sorry to hear that your washing machine is broken In the UK, you can call a professional appliance repair service to fix your washing machine. You can search online for "washing machine repair Southampton" or "appliance repair Southampton" to find local companies that offer this service. Alternatively, you can also check online directories like Rated People or TrustATrader to find a reputable repair service in your area.

CPU times: user 22.2 ms, sys: 5.22 ms, total: 27.4 ms
Wall time: 1.05 s


In [5]:
%%time

getPerplexityResponse(prompt, model='llama-3-70b-instruct', direction=POLITE_DETAIL)

Sorry to hear that your washing machine is broken Don't worry, I'm here to help you get it fixed.

To get your washing machine repaired, you'll need to call an appliance repair technician or a domestic appliance engineer. They specialize in fixing household appliances like washing machines, dishwashers, and refrigerators.

Here are your options:

1. **Manufacturer's Repair Service**: If your washing machine is still under warranty, you can contact the manufacturer's customer service department to schedule a repair. They'll send an authorized technician to fix the issue.
2. **Local Appliance Repair Companies**: Search online for appliance repair companies in Southampton that specialize in washing machine repairs. You can check review websites like TrustATrader, Rated People, or Checkatrade to find reliable and rated technicians in your area.
3. **Independent Repair Technicians**: You can also search for independent repair technicians in Southampton who specialize in washing machine repairs. They might offer more competitive pricing and flexible scheduling.

Before calling a technician, it's a good idea to have some information ready:

* The make and model of your washing machine
* A detailed description of the problem you're experiencing (e.g., not spinning, not draining, not turning on)
* Any error codes or lights that are flashing on the machine

When you call a technician, ask them about their:

* Experience with your washing machine's brand and model
* Availability and response time
* Pricing and payment terms
* Warranty or guarantee on their repairs

Additionally, you can also check if your home insurance policy covers appliance repairs or if you have a home emergency cover that includes washing machine repairs.

I hope this helps you get your washing machine up and running again

CPU times: user 25.3 ms, sys: 5.24 ms, total: 30.5 ms
Wall time: 7.88 s


In [6]:
%%time

getPerplexityResponse(prompt, model='llama-3-sonar-small-32k-chat', direction=POLITE_DETAIL)

I'm so sorry to hear that your washing machine is broken I'd be happy to help you find a solution.

Firstly, let's identify the best course of action. Since you're located in Southampton, UK, I recommend starting with a local approach. Here are a few options you can consider:

1. **Appliance Repair Services**: You can search online for appliance repair services in Southampton that specialize in washing machine repairs. You can check websites like Rated People, TrustATrader, or Checkatrade to find reputable tradespeople in your area. Provide them with your washing machine's make and model, and they'll be able to assess the issue and provide a quote for the repair.
2. **Manufacturer's Customer Service**: If your washing machine is still under warranty or you're unsure about the repair cost, you can contact the manufacturer's customer service department. They may be able to provide guidance on the best course of action or send a technician to repair the machine.
3. **Local Electricians or Handymen**: If you're not sure what type of repair is needed, you can also contact local electricians or handymen in Southampton who may be able to assess the issue and provide a quote for the repair.

As for the trade you need to call, it depends on the nature of the repair. If the issue is electrical, you may want to contact an electrician. If it's a mechanical or technical issue, you may want to contact an appliance repair specialist.

Before calling anyone, it's a good idea to have the following information ready:

* Your washing machine's make and model
* A brief description of the issue you're experiencing
* Any error codes or messages displayed on the machine
* Your location in Southampton (this will help the tradesperson plan their visit)

Remember to always check the tradesperson's reviews, ratings, and qualifications before booking their services. This will ensure you're getting a reliable and skilled professional to fix your washing machine.

I hope this helps you get started on finding a solution for your broken washing machine

CPU times: user 24 ms, sys: 4.85 ms, total: 28.8 ms
Wall time: 4.98 s


Let's see how it handles a query with a more detailed response:

In [ ]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
getPerplexityResponse(prompt, model='llama-3-sonar-small-32k-chat', direction=GREAT_DETAIL)

## 3. Streaming interface

tbd

In [ ]:
%%time

def getPerplexityStreamingResponse(question, model=DEFAULT_MODEL, direction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    print('start')
    response_stream = requests.post(url, json=payload, headers=headers)
    print('end')
    for response in response_stream:
        print(len(response))
    #return response.get("choices")[0].get("message").get("content")

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
#getPerplexityStreamingResponse(prompt, direction=GREAT_DETAIL)